In [2]:
with open("/home/mukul-barhate/Downloads/r.txt", 'r') as file:
    text = file.read()
faqs =text

STEP = 1
sentences = [faqs]
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

2024-06-08 19:34:08.652618: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 19:34:11.775839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
tokenizer = Tokenizer()

In [5]:
tokenizer.fit_on_texts([faqs])

In [6]:
len(tokenizer.word_index)

810

In [7]:
input_sequences = []
for sentence in faqs.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [8]:
input_sequences

[[49, 221],
 [49, 221, 222],
 [49, 221, 222, 113],
 [49, 221, 222, 113, 223],
 [49, 221, 222, 113, 223, 114],
 [49, 221, 222, 113, 223, 114, 40],
 [160, 372],
 [160, 372, 373],
 [160, 372, 373, 114],
 [160, 372, 373, 114, 40],
 [160, 372, 373, 114, 40, 70],
 [160, 372, 373, 114, 40, 70, 25],
 [160, 372, 373, 114, 40, 70, 25, 1],
 [160, 372, 373, 114, 40, 70, 25, 1, 29],
 [14, 1],
 [14, 1, 5],
 [14, 1, 5, 374],
 [14, 1, 5, 374, 16],
 [14, 1, 5, 374, 16, 24],
 [7, 10],
 [7, 10, 6],
 [7, 10, 6, 115],
 [7, 10, 6, 115, 26],
 [7, 10, 6, 115, 26, 4],
 [7, 10, 6, 115, 26, 4, 375],
 [7, 10, 6, 115, 26, 4, 375, 161],
 [71, 1],
 [71, 1, 30],
 [71, 1, 30, 376],
 [71, 1, 30, 376, 13],
 [71, 1, 30, 376, 13, 116],
 [71, 1, 30, 376, 13, 116, 33],
 [71, 1, 30, 376, 13, 116, 33, 2],
 [71, 1, 30, 376, 13, 116, 33, 2, 24],
 [15, 117],
 [15, 117, 224],
 [15, 117, 224, 225],
 [15, 117, 224, 225, 162],
 [15, 117, 224, 225, 162, 118],
 [7, 87],
 [7, 87, 377],
 [7, 87, 377, 1],
 [7, 87, 377, 1, 71],
 [7, 87, 3

In [9]:
max_len = max([len(x) for x in input_sequences])
max_len

19

In [10]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [11]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,  49, 221],
       [  0,   0,   0, ...,  49, 221, 222],
       [  0,   0,   0, ..., 221, 222, 113],
       ...,
       [  0,   0,   0, ..., 107, 108,   5],
       [  0,   0,   0, ..., 108,   5,  52],
       [  0,   0,   0, ...,   5,  52,  36]], dtype=int32)

In [12]:
X = padded_input_sequences[:,:-1]

In [13]:
y = padded_input_sequences[:,-1]

In [14]:
X.shape

(2635, 18)

In [15]:
y.shape

(2635,)

In [16]:
l = y.shape[0]
l

2635

In [17]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=l+1)

In [18]:
y.shape

(2635, 2636)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [21]:
model = Sequential()
model.add(Embedding(l+1, 100, input_length=19))
model.add(LSTM(150))
model.add(Dense(l+1, activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.fit(X,y,epochs=100)

Epoch 1/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - accuracy: 0.0412 - loss: 7.2131
Epoch 2/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.0428 - loss: 5.8589
Epoch 3/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.0436 - loss: 5.7089
Epoch 4/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.0494 - loss: 5.5566
Epoch 5/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.0551 - loss: 5.4316
Epoch 6/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.0720 - loss: 5.3301
Epoch 7/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.0804 - loss: 5.2639
Epoch 8/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.0862 - loss: 5.0510
Epoch 9/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.0868 - loss: 5.0346
Epoch 10/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1068 - loss: 4.9124
Epoch 11/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.1156 - loss: 4.7836
Epoch 12/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step

In [28]:
import time
import numpy as np
text = "what is the fee"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
what is the fee insights
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
what is the fee insights and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
what is the fee insights and perspective
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
what is the fee insights and perspective i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
what is the fee insights and perspective i hadn't
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
what is the fee insights and perspective i hadn't thought
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
what is the fee insights and perspective i hadn't thought of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
what is the fee insights and perspective i hadn't thought of that
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
what is the fee insights and perspective i hadn't thought of that thanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
what is the fee insights and perspective i hadn't thought of that thanks you're


In [26]:
tokenizer.word_index

{'you': 1,
 'to': 2,
 'we': 3,
 'the': 4,
 'for': 5,
 'a': 6,
 'i': 7,
 'and': 8,
 'of': 9,
 'have': 10,
 'our': 11,
 'your': 12,
 'this': 13,
 'thank': 14,
 "i'm": 15,
 'with': 16,
 'in': 17,
 'is': 18,
 "let's": 19,
 'it': 20,
 'on': 21,
 'will': 22,
 'can': 23,
 'me': 24,
 'are': 25,
 'about': 26,
 'any': 27,
 'that': 28,
 'today': 29,
 'please': 30,
 'do': 31,
 'my': 32,
 'topic': 33,
 'us': 34,
 'or': 35,
 'here': 36,
 'some': 37,
 'welcome': 38,
 "it's": 39,
 'name': 40,
 'been': 41,
 "i've": 42,
 'out': 43,
 'take': 44,
 'course': 45,
 'work': 46,
 'event': 47,
 'key': 48,
 'good': 49,
 'time': 50,
 'what': 51,
 'being': 52,
 'am': 53,
 'one': 54,
 'help': 55,
 'point': 56,
 'field': 57,
 'there': 58,
 'not': 59,
 'discuss': 60,
 'hey': 61,
 'see': 62,
 'new': 63,
 'should': 64,
 'like': 65,
 'an': 66,
 'all': 67,
 'be': 68,
 'questions': 69,
 'how': 70,
 'could': 71,
 'interested': 72,
 'research': 73,
 'academic': 74,
 'success': 75,
 'think': 76,
 'feel': 77,
 'as': 78,
 'so'

In [29]:
import tensorflow as tf

In [31]:
model.save('final_model.h5')

In [32]:
model.save('final_model_ker.keras')

In [33]:
import pickle

In [35]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer,f)